# Elektronska fiskalizacija

**Primjeri kod-a u Python-u**

Za primjere u prilogu je neophodno imati `signxml`, pa to valja prethodno instalirati (`!pip install signxml`). Osim toga, podrazumijeva se posjedovanje test PFX sertifikata (_CoreitPecatSoft.pfx_) u lokalnom folderu.

In [1]:
pfx_cert = 'CoreitPecatSoft.pfx'
pfx_password = b'123456'

### Priprema sertifikata

Ekstrakcija privatnog kljuca i sertifikata iz PFX kontejnera. Za taj dio koristim kod dostupan na sljedecem [linku](https://www.jhanley.com/google-cloud-extracting-private-key-from-service-account-p12-credentials/) 

In [2]:
import OpenSSL.crypto
import os

###########################################################    
# Version 1.00
# Date Created: 2018-12-21
# Last Update:  2018-12-21
# https://www.jhanley.com
# Copyright (c) 2018, John J. Hanley
# Author: John Hanley
###########################################################

# Convert a Google P12 (PFX) service account into private key and certificate.
# Convert an SSL Certifcate (PFX) into private key, certificate and CAs.

def write_CAs(filename, p12):
    # Write the Certificate Authorities, if any, to filename

    if os.path.exists(filename):
        os.remove(filename)

    ca = p12.get_ca_certificates()

    if ca is None:
        return

    print('Creating Certificate CA File:', filename)

    with open(filename, 'wb') as f:
        for cert in ca:
            f.write(OpenSSL.crypto.dump_certificate(OpenSSL.crypto.FILETYPE_PEM, cert))

def pfx_to_pem(pfx_path, pfx_password, pkey_path, pem_path, pem_ca_path):
    '''
    Decrypt the P12 (PFX) file and create a private key file and certificate file.

    Input:
        pfx_path    INPUT: This is the Google P12 file or SSL PFX certificate file
        pfx_password    INPUT: Password used to protect P12 (PFX)
        pkey_path   INPUT: File name to write the Private Key to
        pem_path    INPUT: File name to write the Certificate to
        pem_ca_path INPUT: File name to write the Certificate Authorities to
    '''

    print('Opening:', pfx_path)
    with open(pfx_path, 'rb') as f_pfx:
        pfx = f_pfx.read()

    print('Loading P12 (PFX) contents:')
    p12 = OpenSSL.crypto.load_pkcs12(pfx, pfx_password)

    print('Creating Private Key File:', pkey_path)
    with open(pkey_path, 'wb') as f:
        # Write Private Key
        f.write(OpenSSL.crypto.dump_privatekey(OpenSSL.crypto.FILETYPE_PEM, p12.get_privatekey()))

    print('Creating Certificate File:', pem_path)
    with open(pem_path, 'wb') as f:
        # Write Certificate
        f.write(OpenSSL.crypto.dump_certificate(OpenSSL.crypto.FILETYPE_PEM, p12.get_certificate()))

    # Google P12 does not have certifiate authorities but SSL PFX certificates do
    write_CAs(pem_ca_path, p12)


# Start here
pfx_to_pem(
    pfx_cert,         # Google Service Account P12 file
    pfx_password,     # P12 file password
    'coreit.key',      # Filename to write private key
    'coreit_cert.pem', # Filename to write certificate
    'coreit_ca.pem')   # Filename to write CAs if present

Opening: CoreitPecatSoft.pfx
Loading P12 (PFX) contents:
Creating Private Key File: coreit.key
Creating Certificate File: coreit_cert.pem
Creating Certificate CA File: coreit_ca.pem


### Generisanje IKOF-a

In [3]:
import hashlib
from Crypto.PublicKey import RSA
from Crypto.Cipher import PKCS1_v1_5 as Cipher_PKCS1_v1_5
from Crypto.Signature import PKCS1_v1_5
from base64 import b64decode, b64encode
from Crypto.Hash import SHA256

def generate_iis(data, private_key_path):
    message = bytes(data, 'utf8')
    digest = SHA256.new()
    digest.update(message)    
    private_key = False
    with open(private_key_path, "r") as key_file:
        private_key = RSA.importKey(key_file.read())
    signer = PKCS1_v1_5.new(private_key)
    sig = signer.sign(digest)
    iic = hashlib.md5(sig).digest()
    
    return (sig.hex(), iic.hex())

#### Primjer:

In [4]:
(iic, ikof) = generate_iis(
    "{0}|{1}|{2}|{3}|{4}|{5}|{6}".format(
        "12345678",
        "2020-06-12T17:05:43+02:00",
        "9952",
        "bb123bb123",
        "cc123cc123",
        "ss123ss123",
        "99.01"
    ),
    'coreit.key'
)
print("IIC:\n", iic, "\nIKOF:\n", ikof)

IIC:
 46b368a2dd90ec1e70056fa2a5c9116098ac5984bec5fff739beefd1245c0e7d12cdc868ae842b00d36d140a6a5a96f98b37989e548e96122d15122b9240a7bcb9997ae6e8698bdc343f9cf1e84ddd213c5bec4aadb35be9c739fae50a0de6b156429616f971abf211c7a7fa8de8ade10664b52c0b42c2789ca7592c8b1930f4d61a248b7d490f859956f9bc639d502e7434286161d1d26ce2c9ae924d2fa53140bce24f9e51d83f3c0cb555abf5274a18f6333e56f6b6266e5eca664c435d6fc3b87d1337c6b3a45bd391aa2ec13300da79ae1eca858fa8ceb8e3fff882e5c1eb898595ce31f4461b6c83438c4ae0f6ce4ebd162721689a6cf2b9195df4fd59 
IKOF:
 c3145b287db951918e32a0f3e743a635


#### Test:

Za test su poredjene vrijednosti dobijene izvrsavanjem primjera iz Tehnicke specifikacije v3.1

In [5]:
assert iic.upper() == "46B368A2DD90EC1E70056FA2A5C9116098AC5984BEC5FFF739BEEFD1245C0E7D12CDC868AE842B00D36D140A6A5A96F98B37989E548E96122D15122B9240A7BCB9997AE6E8698BDC343F9CF1E84DDD213C5BEC4AADB35BE9C739FAE50A0DE6B156429616F971ABF211C7A7FA8DE8ADE10664B52C0B42C2789CA7592C8B1930F4D61A248B7D490F859956F9BC639D502E7434286161D1D26CE2C9AE924D2FA53140BCE24F9E51D83F3C0CB555ABF5274A18F6333E56F6B6266E5ECA664C435D6FC3B87D1337C6B3A45BD391AA2EC13300DA79AE1ECA858FA8CEB8E3FFF882E5C1EB898595CE31F4461B6C83438C4AE0F6CE4EBD162721689A6CF2B9195DF4FD59"
assert ikof.upper() == "C3145B287DB951918E32A0F3E743A635"

### Priprema i potpisivanje XML-a

Primjer XML fajla u je produzetku. Zbog jednostavnosti se koristim predefinisanim vrijednostima.

Priprema XML objekta i parsiranje XML sadrzaja

In [6]:
xml = '<?xml version="1.0"?><RegisterInvoiceRequest xmlns:xsd="http://www.w3.org/2001/XMLSchema" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" Id="Request" Version="1" xmlns="https://efi.tax.gov.me/fs/schema"><Header UUID="ed6d9696-7d32-4c49-ba28-3fb2a2785a90" SendDateTime="2020-11-02T11:52:13+01:00" /><Invoice TypeOfInv="CASH" IsSimplifiedInv="false" IssueDateTime="2020-11-02T11:52:13+01:00" InvNum="10/2020/nt360fe389" InvOrdNum="10" TCRCode="nt360fe389" IsIssuerInVAT="true" TotPriceWoVAT="10.00" TotPrice="12.10" OperatorCode="oo123oo123" BusinUnitCode="xx123xx123" SoftCode="ss123ss123" IIC="047D11ADAC534EE44D645BF7201B3D1A" IICSignature="6591EF08634CBF6B84C08E6076F3FFD2D93859CC9389F90FAD3F0897569E25ECB3D1DE99246CAB33D049668BC59E17DC56EE4C0A4DAAADB8D3485106F0D2690BE623106C73B1765F39ED8887E3BF4B95671DFE17711CFC004B7DB46DD4EE278A2F0253B206F800650E95CD91FFE473AA571F3427C9DBCF66868C4B993DCBC8E6A4838E25EFC1D0450CBBD7CFA590890437CB51002BC754A201251221415B737F0BDFD37C950AB0400123845E0D6CFC50B87E2C30981A69D68D1981E95862E7F4099D54C296C1E9413DBD9393043EB060DDFB46F620E896A06A5DC5122FE21ABF3BD821854E9F89638C2493491540C68170E9A7A2603CEC6782CF27740CE42670" IsReverseCharge="false"><PayMethods><PayMethod Type="BANKNOTE" Amt="12.10" /></PayMethods><Seller IDType="TIN" IDNum="12345678" Name="Test d.o.o" /><Items><I N="Test artikal" C="1234234" U="kom" Q="10" UPB="1.21" UPA="1.21" PB="10.00" VR="21.00" VA="2.10" PA="12.10" /></Items></Invoice></RegisterInvoiceRequest>'
from lxml import etree
xml_obj = etree.fromstring(xml)

#### Primjer

In [7]:
from lxml import etree
from signxml import XMLSigner, XMLVerifier, methods

cert = open("coreit_cert.pem", 'rb').read()
key = open("coreit.key", 'rb').read()

signer = XMLSigner(
    method = methods.enveloped,
    signature_algorithm = 'rsa-sha256',
    digest_algorithm = 'sha256',
    c14n_algorithm = 'http://www.w3.org/2001/10/xml-exc-c14n#'
)

# Uklanjanje suvisnih DS namespace prefix-a prema preporuci sa: 
# https://github.com/XML-Security/signxml/issues/30#issuecomment-149618525
ns = {}
ns[None] = signer.namespaces['ds']
signer.namespaces = ns

# Konacno potpisivanje
signed_xml = signer.sign(
    xml_obj, 
    key=key,
    cert=cert
)

#### Rezultat

In [8]:
print(etree.tostring(signed_xml, encoding='utf8').decode('utf8'))

<RegisterInvoiceRequest xmlns:xsd="http://www.w3.org/2001/XMLSchema" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xmlns="https://efi.tax.gov.me/fs/schema" Id="Request" Version="1"><Header UUID="ed6d9696-7d32-4c49-ba28-3fb2a2785a90" SendDateTime="2020-11-02T11:52:13+01:00"/><Invoice TypeOfInv="CASH" IsSimplifiedInv="false" IssueDateTime="2020-11-02T11:52:13+01:00" InvNum="10/2020/nt360fe389" InvOrdNum="10" TCRCode="nt360fe389" IsIssuerInVAT="true" TotPriceWoVAT="10.00" TotPrice="12.10" OperatorCode="oo123oo123" BusinUnitCode="xx123xx123" SoftCode="ss123ss123" IIC="047D11ADAC534EE44D645BF7201B3D1A" IICSignature="6591EF08634CBF6B84C08E6076F3FFD2D93859CC9389F90FAD3F0897569E25ECB3D1DE99246CAB33D049668BC59E17DC56EE4C0A4DAAADB8D3485106F0D2690BE623106C73B1765F39ED8887E3BF4B95671DFE17711CFC004B7DB46DD4EE278A2F0253B206F800650E95CD91FFE473AA571F3427C9DBCF66868C4B993DCBC8E6A4838E25EFC1D0450CBBD7CFA590890437CB51002BC754A201251221415B737F0BDFD37C950AB0400123845E0D6CFC50B87E2C30981A69D68D198

#### Test

Za test su koristeni log-ovi iz [Markovog](https://github.com/markoesc) [FiscalizationService](https://github.com/markoesc/FiscalizationService) primjera

In [9]:
signature_value = signed_xml.find('.//{http://www.w3.org/2000/09/xmldsig#}SignatureValue').text
assert signature_value.startswith('N8wdPtLRyfDcJZkLfPvzPvPKeKhP3sPVJ39SZtOLC')
digest_value = signed_xml.find('.//{http://www.w3.org/2000/09/xmldsig#}DigestValue').text
assert digest_value == 'l4IDeuSr6s3qqM7kAkUuIf91nEBcGC81V7wElFl3buA='